In [271]:
import requests
import openai
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import webbrowser
import json
import pandas as pd
import time #dding a delay between requests helps avoid exceeding these limits, preventing errors and potential bans from the API provider.

In [272]:
path = 'Coordinares.csv'
datas = pd.read_csv(path)

In [273]:
# Show all my data
datas[0:18]

,Latitude,Longitude
0,51.47942,-0.29300
1,60.86550,7.11743
2,60.39250,5.34701
3,60.86542,7.11769
4,60.86469,7.11900
5,59.86933,10.71787
6,51.51996,-0.09318
7,59.86905,10.71736
8,60.39449,5.34263
9,51.51952,-0.09428


In [274]:
# OpenCage_key

with open ("OpenCage_key.txt",'r') as key_file:
    OpenCage_key = key_file.read()

In [275]:
# Function Definition,show coodinate to location

def get_location_name(latitude, longitude):
    api_key = OpenCage_key
    url = f'https://api.opencagedata.com/geocode/v1/json?q={latitude}+{longitude}&key={api_key}'
    response = requests.get(url)
    data = response.json()

    if data['results']:
        location = data['results'][0]['formatted']
        components = data['results'][0]['components']
        city = components.get('city', '')
        country = components.get('country', '')
        return location, city, country

latitude = 51.47942
longitude = -0.29300
location, city, country = get_location_name(latitude, longitude)
print(f"Location: {location}, City: {city}, Country: {country}")


Location: Royal Botanic Gardens, Kew, Brentford High Street, London, TW8 0DS, United Kingdom, City: London, Country: United Kingdom


In [276]:
data = pd.read_csv('Coordinares.csv')

In [277]:
#Initializing New Columns

data['Location'] = None
data['City'] = None
data['Country'] = None

#This ensures thatlater populate these columns with data retrieved from the `get_location_name` function, they will exist in the DataFrame.

In [278]:
for index, row in data.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    location, city, country = get_location_name(latitude, longitude)
    
    # Store the retrieved results in the DataFrame
    data.at[index, 'Location'] = location
    data.at[index, 'City'] = city
    data.at[index, 'Country'] = country

In [279]:
#formatted string, index + 1 mean start at 1 not 0 
print(f"Processed {index + 1}/{len(data)}: Location: {location}, City: {city}, Country: {country}")

Processed 18/18: Location: The Tree House, Sayer Street, London, SE17 1FU, United Kingdom, City: London, Country: United Kingdom


In [280]:
print("Data with added location information (first 18 rows):")
print(data[0:18])

Data with added location information (first 18 rows):
    Latitude  Longitude                                           Location  \
0   51.47942   -0.29300  Royal Botanic Gardens, Kew, Brentford High Str...   
1   60.86550    7.11743         5016, Nedre Brekkevegen, 5743 Flåm, Norway   
2   60.39250    5.34701                    Byfjellene, 5019 Bergen, Norway   
3   60.86542    7.11769         5016, Nedre Brekkevegen, 5743 Flåm, Norway   
4   60.86469    7.11900                     A-feltvegen, 5743 Flåm, Norway   
5   59.86933   10.71787        Langøyene, German Bunker, 0252 Oslo, Norway   
6   51.51996   -0.09318  Frobisher Crescent, Barbican, London, EC2Y 8HD...   
7   59.86905   10.71736        Langøyene, German Bunker, 0252 Oslo, Norway   
8   60.39449    5.34263   Martensgården, unnamed road, 5019 Bergen, Norway   
9   51.51952   -0.09428  Defoe House, Lakeside Terrace, Barbican, Londo...   
10  51.46190   -0.02903  Hilly Fields Crescent, London, SE4 1QA, United...   
11  51.502

In [281]:
#new csv file included the locations
data.to_csv('updated_locations.csv', index=False)
print("Data with location information has been saved to updated_locations.csv")

Data with location information has been saved to updated_locations.csv


In [282]:
def get_access_token():
    # Load credentials from JSON file
    with open('spotify_keys.json') as f:
        spotify_keys = json.load(f)

    # Use SpotifyOAuth to generate an access token
    auth_manager = SpotifyOAuth(
        client_id=spotify_keys["client_id"],
        client_secret=spotify_keys["client_secret"],
        redirect_uri=spotify_keys["redirect"],
        scope="playlist-modify-public"
    )
    
    # Create a Spotipy instance and return it
    sp = spotipy.Spotify(auth_manager=auth_manager)
    return sp

# Get the Spotify client instance
sp = get_access_token()

# Verify if the token was successfully obtained
user_id = sp.current_user()['id']
print(f"Successfully authenticated as {user_id}")

Successfully authenticated as 314qipe2nxyhqxkqddeo36x4mk7i


In [283]:
# Load location data from CSV file
data = pd.read_csv('updated_locations.csv')

In [284]:
pd.DataFrame(data)

,Latitude,Longitude,Location,City,Country
0,51.47942,-0.29300,"Royal Botanic Gardens, Kew, Brentford High Str...",London,United Kingdom
1,60.86550,7.11743,"5016, Nedre Brekkevegen, 5743 Flåm, Norway",NaN,Norway
2,60.39250,5.34701,"Byfjellene, 5019 Bergen, Norway",Bergen,Norway
3,60.86542,7.11769,"5016, Nedre Brekkevegen, 5743 Flåm, Norway",NaN,Norway
4,60.86469,7.11900,"A-feltvegen, 5743 Flåm, Norway",NaN,Norway
5,59.86933,10.71787,"Langøyene, German Bunker, 0252 Oslo, Norway",Oslo,Norway
6,51.51996,-0.09318,"Frobisher Crescent, Barbican, London, EC2Y 8HD...",London,United Kingdom
7,59.86905,10.71736,"Langøyene, German Bunker, 0252 Oslo, Norway",Oslo,Norway
8,60.39449,5.34263,"Martensgården, unnamed road, 5019 Bergen, Norway",Bergen,Norway
9,51.51952,-0.09428,"Defoe House, Lakeside Terrace, Barbican, Londo...",London,United Kingdom


In [285]:
def create_playlist_for_location(sp, latitude, longitude):
    user_id = sp.current_user()['id']  # Get the user ID
    playlist_name = f"Playlist for {latitude}, {longitude}"
    playlist = sp.user_playlist_create(
        user=user_id,
        name=playlist_name,
        public=True,
        description=f"Music inspired by location {latitude}, {longitude}"
    )
    
    print(f"Created playlist: {playlist['name']} - {playlist['external_urls']['spotify']}")
    return playlist

In [286]:
for index, row in data.iterrows():
    address = row['Location']  # Use the 'Location' column to get the address
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    try:
        # Create a playlist
        playlist = create_playlist_for_location(sp, latitude, longitude)

        # Search for music
        search_query = f"{address} artist"
        results = sp.search(q=search_query, type="track", limit=10)

        # Check search results
        if 'tracks' in results and 'items' in results['tracks']:
            tracks = [track['uri'] for track in results['tracks']['items']]
        
            # Add tracks to the playlist
            if tracks:
                sp.playlist_add_items(playlist['id'], tracks)
                print(f"Added {len(tracks)} tracks to playlist for {latitude}, {longitude}")
            else:
                print(f"No tracks found for address {address}")
        else:
            print(f"Unexpected search results for address {address}")

    except Exception as e:
        print(f"An error occurred for address {address}: {e}")

    # Add a delay to avoid request rate limiting
    time.sleep(1)

Created playlist: Playlist for 51.47942, -0.293 - https://open.spotify.com/playlist/2sCD7z0GPJLjDnjyn4JjGN
Added 10 tracks to playlist for 51.47942, -0.293
Created playlist: Playlist for 60.8655, 7.11743 - https://open.spotify.com/playlist/0oMbUlyWNyCl9dieiJaAGG
Added 10 tracks to playlist for 60.8655, 7.11743
Created playlist: Playlist for 60.3925, 5.34701 - https://open.spotify.com/playlist/65BQpZ134qO0zyhN0DMQ7q
Added 10 tracks to playlist for 60.3925, 5.34701
Created playlist: Playlist for 60.86542, 7.11769 - https://open.spotify.com/playlist/0e0BaNJNFAKJpXb3ag1zbc
Added 10 tracks to playlist for 60.86542, 7.11769
Created playlist: Playlist for 60.86469, 7.119 - https://open.spotify.com/playlist/5X0j9v3gujmbZJ1YPUSLix
Added 10 tracks to playlist for 60.86469, 7.119
Created playlist: Playlist for 59.86933, 10.71787 - https://open.spotify.com/playlist/03msP6A5pCmJ9ijC5MNHfF
Added 10 tracks to playlist for 59.86933, 10.71787
Created playlist: Playlist for 51.51996, -0.09318 - https://

In [266]:
playlist_url = "https://open.spotify.com/playlist/YOUR_PLAYLIST_ID"
webbrowser.open(playlist_url)

True

In [237]:
#Using geoapify 

In [268]:
#This line defines a function called `search_place` that takes one parameter, `location_name`.
#This parameter represents the name of the location you want to search for.

def search_place(location_name):
    with open("Geoapify_key.txt", 'r') as key_file:
        geoapify_key = key_file.read().strip()
    
    # Use the Geoapify API to search for the location
    url = f"https://api.geoapify.com/v1/geocode/search?text={location_name}&apiKey={geoapify_key}"
    
    response = requests.get(url)
    
    #This means the request was successful, and the server has returned the requested resource.
    if response.status_code == 200:
        data = response.json()
        if 'features' in data and len(data['features']) > 0:
            # Return the place ID of the first result
            place_id = data['features'][0]['properties']['place_id']
            return place_id
        else:
            print("No results found for the location.")
            return None
    

# Example: Get the Place ID for a specific location
location_name = "Royal Botanic Gardens, Kew" 
place_id = search_place(location_name)

if place_id:
    print(f"Place ID for '{location_name}': {place_id}")


Place ID for 'Royal Botanic Gardens, Kew': 510c9e04449ec1d2bf59b8e2e2a8dcbd4940f00102f9018755490000000000c0020192030c546865204f72616e67657279


In [269]:
def extract_keywords(place_details):
    keywords = set()  # Use a set to avoid duplicates

    # Define a set of unwanted keywords
    unwanted_keywords = {"yes", "no", "true", "false", "null"}

    if place_details and 'features' in place_details:
        for feature in place_details['features']:
            categories = feature['properties'].get('categories', [])
            # Extract keywords from categories
            for category in categories:
                keyword = category.split('.')[-1]  # Get the last part as keyword
                if keyword not in unwanted_keywords:  # Check against unwanted keywords
                    keywords.add(keyword)

            # Add the name as a keyword if available
            if 'name' in feature['properties']:
                name = feature['properties']['name']
                if name not in unwanted_keywords:  # Check against unwanted keywords
                    keywords.add(name)

    return keywords

# Get place details (assuming place_id is defined)
place_details = get_place_details(place_id)

if place_details:
    keywords = extract_keywords(place_details)
    
    # Ensure keywords is a set before joining
    if isinstance(keywords, set) and keywords:
        print("Extracted Keywords:", ', '.join(keywords))

Extracted Keywords: restaurant, wheelchair, vegan, The Orangery, catering, building, vegetarian


In [270]:
# Function to search and add tracks to the playlist
def search_and_add_tracks(sp, playlist_id, keywords):
    search_query = ' '.join(keywords)  # Combine keywords into a single search string
    print(f"Searching for tracks with keywords: {search_query}")
    results = sp.search(q=search_query, type='track', limit=10)

    if 'tracks' in results and 'items' in results['tracks']:
        tracks = [track['uri'] for track in results['tracks']['items']]
        if tracks:
            sp.playlist_add_items(playlist_id, tracks)
            print(f"Added {len(tracks)} tracks to playlist for keywords: {search_query}")
        else:
            print("No tracks found for the keywords.")
    else:
        print("Unexpected search results.")

playlist_id = create_playlist(sp, latitude, longitude)

# Search for music and add it to the playlist
search_and_add_tracks(sp, playlist_id, keywords)



Created playlist: Playlist for 51.49312, -0.0971 - https://open.spotify.com/playlist/7zqZdlycEr8lLYCZS9hLVb
Searching for tracks with keywords: restaurant wheelchair vegan The Orangery catering building vegetarian
Added 10 tracks to playlist for keywords: restaurant wheelchair vegan The Orangery catering building vegetarian
